In [1]:
import asyncio

import spotipy
import spotipy.util as util
from spotipy import SpotifyClientCredentials
from eda.spotify.jojo.get_artist_to_uris import REPO_ROOT
import api_setup
import pandas as pd
from aiohttp import ClientSession
import requests
import base64
import json

ModuleNotFoundError: No module named 'tekore'

In [14]:

# API Auth
env_vars = api_setup.parse_api_kvs(REPO_ROOT / "api-keys")
auth_manager = SpotifyClientCredentials(env_vars['client_id'], env_vars['client_secret'])
spotify = spotipy.Spotify(client_credentials_manager=auth_manager, backoff_factor=2)

In [15]:
# Load in data
uri_df = pd.read_json("artist_track_to_uri.json")
uris = uri_df.uri.values
uris

array(['spotify:track:375h23169u0dcEho5hEkML',
       'spotify:track:6FRqV0YP0tyazXOIxexjW5',
       'spotify:track:4wYgevsnMVdxtHTSjboCJ7', ...,
       'spotify:track:63Kl7faGbITZxkJGtO7MPT',
       'spotify:track:7F8LoNeDRgItTcpTpYTmY3',
       'spotify:track:5W1qeEGfh4SaUxniF9VKP6'], dtype=object)

In [30]:
# async api calls
async def get_audio_features(session: ClientSession, uri: str) -> dict:
	# Trim the input string - we don't want the 'spotify:track:` part
	uri = uri[14:]
	endpoint = f"https://api.spotify.com/v1/audio-features/{uri}"

	async with session.get(endpoint) as response:
		response = await(response.json())
		return uri, response

async def main(startindex: int=0, endindex: int=-1):
	creds = f"{env_vars['client_id']}:{env_vars['client_secret']}"
	creds_b64 = base64.b64encode(creds.encode())
	headers= {"Authorization": f"Basic {creds_b64.decode()}"}
	data= {"grant_type": "client_credentials"}
	token = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
	token = token.json()['access_token']
	headers = {"Accept": "application/json", "Content-Type": "application/json", "Authorization": f"Bearer {token}"}
	async with ClientSession(headers=headers) as session:
		batch_uris = uris[startindex:endindex]
		tasks = []
		for uri in batch_uris:
			task = asyncio.ensure_future(get_audio_features(session, uri))
			tasks.append(task)

		features = await(asyncio.gather(*tasks))
	return features

features = await(main(0, 5000))

In [ ]:
features = [feature[1] for feature in features]

In [36]:
with open("features_0.json", "w") as f:
	json.dump(features, f)

In [37]:
print(len(features))

5000
